---
title: Helene Photogrametery
description: This tutorial demonstrates how download images from ArcGIS Server into GRASS GIS.
format: html
author: Corey T. White
date: '2024-10-9'
keep-ipynb: true
toc: true
toc-depth: 4
engine: jupyter
execute:
  eval: false
jupyter: python3
image: images/DSC_0067.JPG
categories: [geospatial, GRASS, jupyter, helene]
page-layout: full
title-block-banner: true
draft: true
---


# Download Flight Data

## Set up environment

In [1]:
import sys
import subprocess
from pathlib import Path
from IPython.display import display

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

# Append GRASS to the python system path
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS libraries
import grass.script as gs
import grass.jupyter as gj

# GRASS Variables
grassdata = "grassdata"
project_name = "Swannanoa"
mapset = "flight"

## GRASS

### Create project


In [2]:
#| eval: false
gs.create_project(path=grassdata, name=project_name, epsg="3358", overwrite=False)

### Start GRASS session


In [3]:
# Start GRASS in the recently created project
session = gj.init(Path(grassdata, project_name))

### Define Query Parameters

In [4]:
project = '`CAP - H-TS Helene 2024`' # Queried at API level
mission = "'24-1-5849'"
flight = "'A0046B'" # Swannanowa

url = f"https://services.arcgis.com/XG15cJAlne2vxtgt/ArcGIS/rest/services/Image_Points_view/FeatureServer/3/query?where=project+%3D+%27CAP+-+H-TS+Helene+2024%27+and+mission+%3D+%2724-1-5849%27+and+flight+%3D+%27{flight}%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=3358&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=geojson&token="

### Import Data


In [5]:
#| eval: false
gs.run_command("v.import", input=url, output="image_points", overwrite=True)

### Get data info


In [6]:
gs.run_command("v.info", map="image_points", flags="c" )

### Set region


In [7]:
gs.run_command("g.region", vector="image_points", res=10)

### View Data


In [8]:
m = gj.Map(use_region=True)
m.d_vect(map="image_points")
display(m.show())

### Read data into pandas DataFrame


In [9]:
import pandas as pd

points_json = gs.parse_command("v.db.select",
    map="image_points",
    format="json",
    flags=""
)

points_df = pd.DataFrame(points_json["records"])
points_df.head()

## Download images for processing


In [10]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

def query_feature_service(flight_id):
    try:
        rest_url = 'https://services.arcgis.com/XG15cJAlne2vxtgt/ArcGIS/rest/services/Image_Points_view/FeatureServer/3/query'

        query = f"project='CAP - H-TS Helene 2024' AND mission='24-1-5849' AND flight='{flight_id}'"

        params = {
            'where': query,
            'outFields': '*',
            'f': 'json',
            'returnGeometry': 'false'
        }
        response = requests.get(rest_url, params=params)
        json_data = response.json()
        url_list = list(map(lambda feat: feat.get('url'), json_data['features']))
        return url_list
    except requests.RequestException as e:
        print(f"Error fetching data from FeatureService: {e}")

# Function to download an image from a URL
def download_image(url, save_path, session):
    try:
        response = session.get(url)
        response.raise_for_status()  # Check for HTTP errors
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {url} to {save_path}")
    except requests.RequestException as e:
        print(f"Failed to download {url}: {e}")

# Main function to set up the thread pool and download images
def run(urls, save_dir, num_threads=4):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        with requests.Session() as session:
            futures = []
            for url in urls:
                filename = os.path.basename(url)
                save_path = os.path.join(save_dir, filename)
                futures.append(executor.submit(download_image, url, save_path, session))

            for future in as_completed(futures):
                future.result()  # This will raise 

### Run the download command

Make sure to set appropriate threads for your system.

In [11]:
#| eval: false
url_list = list(points_df.url.values)

run(urls=url_list, save_dir="imagery_data/nadir/{flight}", num_threads=32)

### Example output

| | | |
|-|-|-
|![](images/CAP_2147.JPG)|![](images/CAP_2148.JPG)|![](images/CAP_2149.JPG)
|![](images/CAP_2150.JPG)|![](images/CAP_2151.JPG)|![](images/CAP_2152.JPG)
|![](images/CAP_2153.JPG)|![](images/CAP_2154.JPG)|![](images/CAP_2155.JPG)


<!-- ## WIP -->

### Run data in your favorite photogrametry software

- WebODM
- Agisoft Metashape


In [12]:
#| echo: false
#| eval: false

image_urls = query_feature_service(flight_id="A0046B")